# Projeto de Automação de envio de e-mails com indicadores de Lojas 

In [4]:
# Importar bibliotecas:
import pandas as pd
import pathlib
import win32com.client as win32

# Importar bases de dados:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

# Incluir o nome da loja na tabela vendas:
vendas = vendas.merge(lojas, on='ID Loja')

# Criar uma tabela para cada loja:
dicionario_lojas = {}
for loja in lojas['Loja']:
    # Criando cada uma das lojas com o for:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

# Criando o indicador de dia:
dia_indicador = vendas['Data'].max()

# Identificar se a pasta já existe:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivos_pasta_backup = caminho_backup.iterdir()

# Fazendo o List Comprehension do for comentado abaixo:
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja  
        nova_pasta.mkdir()
    
# Salvar dentro da pasta:
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo) 
    
# Definição de Metas:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    # Faturamento:
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
 
    # Diversidade de Produtos:
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket Médio:
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    # Enviar o e-mail:
    outlook = win32.Dispatch('outlook.application')
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia: {dia_indicador.day}/{dia_indicador.month} - Loja: {loja}.'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia ='red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    mail.HTMLBody = f'''
    <p>Bom dia, <b>{nome}</b></p>

    <p>O resultado de ontem, dia <strong>{dia_indicador.day}/{dia_indicador.month}</strong>, da <strong>Loja: {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:,.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:,.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:,.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:,.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:,.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:,.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:,.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>
    
    <p>Segue anexa planilha com todos os dados para mais detalhes.</p>
    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Lucas Carvalho.</p>
    '''

    # Anexos (Pode colocar quantos quiser):
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print(f'E-mail da loja {loja} enviado')
    
# Criando os Rankings para a Diretoria:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
#display(faturamento_lojas_ano)

# Criando o ranking anual em excel:
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'  
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
#display(faturamento_lojas_dia)

# Criando o ranking diário em excel:
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Diário.xlsx'  
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

# Enviar o e-mail para a Diretoria:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Anual e Diário: {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia!

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]}, com faturamento R${faturamento_lojas_dia.iloc[0, 0]:,.2f}.
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]}, com faturamento R${faturamento_lojas_dia.iloc[-1, 0]:,.2f}.

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]}, com faturamento R${faturamento_lojas_ano.iloc[0, 0]:,.2f}.
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[-1]}, com faturamento R${faturamento_lojas_ano.iloc[-1, 0]:,.2f}.

Seguem anexos os rankings do ano e dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att, 
Lucas Carvalho. 
'''

# Anexos (Pode colocar quantos quiser):
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Diário.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print(f'E-mail da Diretoria enviado.')



E-mail da loja Iguatemi Esplanada enviado
E-mail da loja Shopping Midway Mall enviado
E-mail da loja Norte Shopping enviado
E-mail da loja Shopping Iguatemi Fortaleza enviado
E-mail da loja Shopping União de Osasco enviado
E-mail da loja Shopping Center Interlagos enviado
E-mail da loja Rio Mar Recife enviado
E-mail da loja Salvador Shopping enviado
E-mail da loja Rio Mar Shopping Fortaleza enviado
E-mail da loja Shopping Center Leste Aricanduva enviado
E-mail da loja Ribeirão Shopping enviado
E-mail da loja Shopping Morumbi enviado
E-mail da loja Parque Dom Pedro Shopping enviado
E-mail da loja Bourbon Shopping SP enviado
E-mail da loja Palladium Shopping Curitiba enviado
E-mail da loja Passeio das Águas Shopping enviado
E-mail da loja Center Shopping Uberlândia enviado
E-mail da loja Shopping Recife enviado
E-mail da loja Shopping Vila Velha enviado
E-mail da loja Shopping SP Market enviado
E-mail da loja Shopping Eldorado enviado
E-mail da loja Shopping Ibirapuera enviado
E-mail da 